In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests

from time import sleep
import pandas as pd 
import numpy as np

In [2]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [3]:
# get main shelter that sit on top of parent shelters => Main > Parent > Child

driver.get("https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/accueil.aspx")
page = driver.find_element_by_class_name('divWCAD304_FamilyList')


parent_urls = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
parent_urls = parent_urls[2:10]
urls = []
for parent in parent_urls:
    sleep(1)
    driver.get(parent)
    sleep(1)
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "divWCAD304_FamilyList")))
        page = driver.find_element_by_class_name('divWCAD304_FamilyList')
        a = [elem.get_attribute("href") for elem in page.find_elements_by_tag_name('a')]
        for url in a:
            urls.append(url)
    except:
        next

        
urls
    


['https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284327-Poissonnerie.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-285187-Traiteur-de-la-mer.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284351-Legumes.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284350-Jus-de-fruits-et-legumes-frais.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284349-Fruits.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284358-Pains-et-Patisseries.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284361-Pains-de-mie-et-Pains-grilles.aspx',
 'https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284362-Patisse

In [10]:
## test
url = "https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx"
driver.get(url)
driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-2].text[:-2]

'Viandes Poissons'

In [4]:
product_pages = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'}

for url in urls:
    driver.get(url)

    shelters = driver.find_element_by_class_name('divWCRS315_FiltresNavigation').find_element_by_css_selector('ul')
    shelter_main = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-2].text[:-2]
    shelter_parent = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text
    links = [elem.get_attribute("href") for elem in shelters.find_elements_by_tag_name('a')]


    for link in links:
        sleep(1)
        if links.index(link) == 0:
            next
        else:      
            print(link)
            driver.get(link)

            actions = ActionChains(driver)
            actions.move_to_element(driver.find_element_by_id("ulListeProduits").find_elements_by_css_selector('li')[-1]).perform()

            shelter_child = driver.find_element_by_class_name('ulWCAD307_FilAriane').find_elements_by_css_selector('li')[-1].text

            request_text = requests.get(link, headers=headers)
            soup = BeautifulSoup(request_text.text,"html.parser" ,from_encoding="utf-8")


            brands = []
            try:
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of_element_located((By.XPATH, "//dt[contains(text(),'Marques')]")))
                if driver.find_element_by_id('divBlocsFiltres').find_element_by_xpath("//dt[contains(text(),'Marques')]"):
                    for filter in driver.find_elements_by_class_name('dlCRS315_Filtre'):
                        if "Marques" in filter.find_element_by_css_selector("dt").text:
                            for label in filter.find_elements_by_css_selector('label'):
                                brands.append(label.text)
                product_pages.append({"url":link, 
                         "shelter_main": shelter_main,
                         "shelter_parent": shelter_parent,
                         "shelter_child": shelter_child,
                         "brands":brands,
                         "soup": soup, 
                         "text": driver.find_element_by_id("ulListeProduits").text
                        })
            except:
                product_pages.append({"url":link,
                     "shelter_main": shelter_main,
                     "shelter_parent": shelter_parent,
                     "shelter_child": shelter_child,
                     "brands":brands,
                     "soup": soup,
                     "text": driver.find_element_by_id("ulListeProduits").text
                    })


https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284328


/anaconda3/lib/python3.7/site-packages/bs4/__init__.py:191: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284329
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284330
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284331
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284332
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284334
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284325-Boucherie.aspx?Filtres=4-284335
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx?Filtres=4-284336
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx?Filtres=4-284337
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284326-Volailles-et-Gibiers.aspx?Filtres=4-284339
http

https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284373-Charcuteries.aspx?Filtres=4-284397
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284373-Charcuteries.aspx?Filtres=4-284952
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284399
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284404
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284400
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284408
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284401
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284406
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284374-Traiteur.aspx?Filtres=4-284407
https://fd4-courses.leclercdrive.fr/ma

https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284423-Plats-cuisines.aspx?Filtres=4-284444
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284423-Plats-cuisines.aspx?Filtres=4-284445
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-284447
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-287899
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-284448
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-284449
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-284450
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284424-Aperitifs.aspx?Filtres=4-284451
https://fd4-courses.leclercdrive.fr/magasin-127811-Bois-dArcy/rayon-284425-Assaisonnements-et-Condiments.aspx?Filtres=4-284453
https:/

IndexError: list index out of range

In [5]:
data = pd.DataFrame(product_pages)
#data.to_csv('leclerc_catalog.csv')

In [6]:
import html
pages = []
for index, row in data.iterrows():
    soup = row.soup
    page = html.unescape(soup.text)
    page.find('objElement')
    page.find('lstAssocFiltreProduit')
    page = page[(page.find('objElement')-2):(page.find('lstAssocFiltreProduit')-2)]
    page = page.split("objElement")
    pages.append({"shelter_main": row.shelter_main,
                  "shelter_parent": row.shelter_parent,
                  "shelter_child": row.shelter_child,                  
                  "brands": row.brands,
                  "page": page[1:-1]
                })

In [7]:
products = []
for page in pages:
    for content in page['page']:
        item = content.split(",")
        for line in item:
            if "sLibelleLigne1" in line:
                description1 = line[18:-1]
            if "sLibelleLigne2" in line:
                description2 = line[18:-1]
            if "sPrixUnitaire" in line:
                price = line[17:-1]
            if "sPrixParUniteDeMesure" in line:
                price_per_unit = line[25:-1]
            if "sPrixPromo" in line:
                promo_price = line[14:-1]
            if "sUrlVignetteProduit" in line:
                image_url = line[23:-1]
            if "sOrigineZoneDepeche" in line:
                origin = line[23:-1]
            if "sUrlPageProduit" in line:
                url = line[19:-1]
            if "iIdProduit" in line:
                product_id = line[13:]
        description = description1 + " " + description2
        if len(page['brands']) > 0:
            brand = ''.join([brand for brand in page['brands'] if brand.lower() in description.lower()])
        else:
            brand = ""
                
        products.append({"shelter_main": page['shelter_main'],
                         "shelter_parent": page['shelter_parent'],
                         "shelter_child": page['shelter_child'],  
                         "description1":description1, 
                         "description2":description2,                          
                         "brand": brand,
                         "price": price,
                         "price_per_unit":price_per_unit,
                         "offer_description": promo_price,
                         "origin":origin,
                         "image_url": image_url,                    
                         "url": url,
                         "product_id": product_id
                        })


In [8]:
product_data = pd.DataFrame(products)

export_csv = product_data.to_csv (r'/Users/vincentsalamand/Downloads/products_leclerc.csv', index = None, header=True)


In [9]:
# Select duplicate rows except first occurrence based on all columns
duplicateRowsDF = product_data[product_data.duplicated()]
 
print("Duplicate Rows except first occurrence based on all columns are :")
print(duplicateRowsDF)

Duplicate Rows except first occurrence based on all columns are :
Empty DataFrame
Columns: [shelter_main, shelter_parent, shelter_child, description1, description2, brand, price, price_per_unit, offer_description, origin, image_url, url, product_id]
Index: []
